In [1]:
#we load our libraries
import pandas as pd
import sys
sys.path.append('/Users/halukamb/Documents/CodeRelated/HBDataviz_new/2025/01_BTW100Deutsche/Wrangle/lib/python3.12/site-packages')
import numpy as np
from json import loads, dumps

In [2]:
! pip install pyreadstat


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
#we have one function that filters the ALLBUS data for whether respondents are german and of voting age
def germanElectorate(df):
    german_df=df[df["german"]=="JA"]
    german_df=german_df.dropna(subset="age")
    german_df["age"]=german_df["age"].astype(float)
    german_electorate=german_df[german_df["age"]>=18]
    return german_electorate

In [4]:
#we have another function that regroups the data into fitting categories to the data we have from the zensus
def reformgroups(df):
    df["Alter"]=None
    df["Alter"]=np.where(df["age"]<30,"U30",df["Alter"])
    df["Alter"]=np.where(df["age"]>=30,"30-49",df["Alter"])
    df["Alter"]=np.where(df["age"]>=50,"50-64",df["Alter"])
    df["Alter"]=np.where(df["age"]>64,"Ü65",df["Alter"])
    df["Geschlecht"]=np.nan
    df["Geschlecht"]=np.where(df["sex"]=="FRAU","Weiblich",df["Geschlecht"])
    df["Geschlecht"]=np.where(df["sex"]=="MANN","Männlich",df["Geschlecht"])
    df["OstWest"]=np.nan
    df["OstWest"]=np.where(df["eastwest"]=="ALTE BUNDESLAENDER","West",df["OstWest"])
    df["OstWest"]=np.where(df["eastwest"]=="NEUE BUNDESLAENDER","Ost",df["OstWest"])
    df["OstWest"]=np.where(df["land"].str.contains("BERLIN"),"Berlin",df["OstWest"])

    df["Besiedelung"]=np.nan
    df["Besiedelung"]=np.where(df["gs01"].isin(['GROSSSTADT']),"dicht besiedelt",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['MITTEL-, KLEINSTADT', 'VORORT GROSSSTADT']),"mittlere Besiedlungsdichte",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['LAENDL. DORF','EINZELHAUS, LAND']),"gering besiedelt",df["Besiedelung"])
    
    


    
    return df

In [5]:
#and lastly we have a stratification function that takes the zensus data and the skewed dataset of allbus
#  and creates a stratified dataset based on the population shares of the zensus data
#for each combination of age, gender, eastwest and population density
def stratifier(zensus_model,skewed_df):

    stratified_dataset=pd.DataFrame()
    samplesize_sum=0

    #we go through each row of the zensus groups
    for index, row in zensus_model.iterrows():
        #take in which share of the population this group represents
        samplesize = round(row["Share"] * 100)
        samplesize_sum += samplesize
        #and then we take a copy of the skewed allbus data
        sub_df = skewed_df.copy()
        
        for i in range(0, 4):
            #and then we filter it down, feature by feature of the speficic row of the zensus data
            #we do this from 0 to 4 because in the first four rows we have the features we want to stratify by
            sub_df = sub_df[sub_df[row.index[i]] == row.values[i]]
            print(row.index[i],row.values[i],len(sub_df))
        
        
        #we draw a sample from this group that correlates to the share of the population
        #if the original dataset is too small, we sample with replacement
        sampled_subgroup = sub_df.sample(samplesize, random_state=1, replace=True)
        #and add this sampled group to the overall stratified dataset
        stratified_dataset = pd.concat([stratified_dataset, sampled_subgroup])
    print(len(stratified_dataset),samplesize_sum)
    return stratified_dataset


In [6]:
#this is the zensus data we need, we remove all rows with NaN values
#as we need as detailed data as possible for the stratification
zensus_model=pd.read_csv("zensus_model.csv")
zensus_model=zensus_model.dropna()
zensus_model=zensus_model.sort_values(by="Share",ascending=False)


In [7]:
#we read in the allbus data for 2021
df_2021 = pd.read_spss("../data/AllbusData/ZA5280_v2-0-1.sav")
# filter it for the german electorate

df_2021 = germanElectorate(df_2021)
# regroup it to fit the zensus data

df_2021= reformgroups(df_2021)
# and stratify it based on the zensus data
df_2021= stratifier(zensus_model,df_2021)
df_2021

Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter Ü65 223
Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter 50-64 247
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter 50-64 233
Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter 30-49 193
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter 30-49 176
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter Ü65 263
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter Ü65 60
Geschlecht Männlich 2419
Besiedelung dicht besiedelt 451
OstWest West 247
Alter 30-49 79
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter 30-49 83
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter 50-64 60
Geschlecht Männlich 2419
Besiedelung d

,za_nr,doi,version,respid,substudy,mode,splt21,eastwest,german,ep01,...,bik,gkpol,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
986,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),987.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 1 ;>499.999,100.000 - 499.999 E.,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3459,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),3460.0,SIMULTAN,MAIL,SPLIT C,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 1 ;<500.000,100.000 - 499.999 E.,ALTE BUNDESLAENDER,NaN,NaN,NaN,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1790,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1791.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,GUT,...,ZONE 2-4;>499.999,20.000 - 49.999 EINW,ALTE BUNDESLAENDER,0.842875,ALTE BUNDESLAENDER,1.041708,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3381,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),3382.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,Ü65,Weiblich,West,mittlere Besiedlungsdichte
4800,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4801.0,SEQUENZIELL,CAWI,SPLIT C,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 2-4;<500.000,5.000 - 19.999 EINW,ALTE BUNDESLAENDER,0.842875,ALTE BUNDESLAENDER,1.041708,Ü65,Weiblich,West,mittlere Besiedlungsdichte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4711.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;<100.000,2.000 - 4.999 EINW,NEUE BUNDESLAENDER,0.267321,NEUE BUNDESLAENDER,0.141565,U30,Weiblich,Ost,gering besiedelt
1464,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1465.0,SIMULTAN,MAIL,SPLIT B,NEUE BUNDESLAENDER,JA,SCHLECHT,...,ZONE 2-4;<500.000,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Weiblich,Ost,gering besiedelt
4710,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4711.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;<100.000,2.000 - 4.999 EINW,NEUE BUNDESLAENDER,0.267321,NEUE BUNDESLAENDER,0.141565,U30,Weiblich,Ost,gering besiedelt
2291,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),2292.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;>499.999,5.000 - 19.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Weiblich,Ost,gering besiedelt


In [8]:
#we read in the allbus data for 2023
df_2023 = pd.read_spss("../data/AllbusData/ZA8830_v1-0-0.sav")
# filter it for the german electorate

df_2023 = germanElectorate(df_2023)
# regroup it to fit the zensus data

df_2023= reformgroups(df_2023)
#and straitify it based on the zensus data
df_2023= stratifier(zensus_model,df_2023)
df_2023


Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter Ü65 208
Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter 50-64 230
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter 50-64 184
Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter 30-49 197
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter 30-49 178
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter Ü65 222
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter Ü65 76
Geschlecht Männlich 2292
Besiedelung dicht besiedelt 471
OstWest West 257
Alter 30-49 85
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter 30-49 60
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter 50-64 64
Geschlecht Männlich 2292
Besiedelung d

,za_nr,doi,version,respid,substudy,mode,splt23_1,splt23_2,eastwest,german,...,bik,xs11,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
770,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",771.0,NaN,CAPI,NaN,SPLIT A,ALTE BUNDESLAENDER,JA,...,ZONE 2-4;>499.999,183.0,1.252326,1.69639,1.242308,2.107439,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3538,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3539.0,NaN,CAPI,NaN,SPLIT B,ALTE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,93.0,1.252326,0.848195,1.242308,1.053719,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1776,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",1777.0,SIMULTAN,MAIL,SPLIT B,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 1-4;<50.000,10.0,1.203727,1.693259,1.192166,2.018646,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3432,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3433.0,SEQUENZIELL,CAWI,SPLIT A,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 1 ;<500.000,168.0,1.207467,0.554394,1.205184,0.668147,Ü65,Weiblich,West,mittlere Besiedlungsdichte
5150,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",5151.0,NaN,CAPI,NaN,SPLIT B,ALTE BUNDESLAENDER,JA,...,ZONE 2-4;>499.999,143.0,1.252326,0.848195,1.242308,1.053719,Ü65,Weiblich,West,mittlere Besiedlungsdichte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1897,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",1898.0,NaN,CAPI,NaN,SPLIT B,NEUE BUNDESLAENDER,JA,...,5.000-19.999 EINW.,55.0,0.501346,0.532744,0.524886,0.27963,U30,Weiblich,Ost,gering besiedelt
3272,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3273.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 1 ;<500.000,176.0,0.544816,0.845091,0.572064,0.483446,U30,Weiblich,Ost,gering besiedelt
4876,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",4877.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 2-4;<500.000,31.0,0.544816,0.338036,0.572064,0.193378,U30,Weiblich,Ost,gering besiedelt
4876,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",4877.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 2-4;<500.000,31.0,0.544816,0.338036,0.572064,0.193378,U30,Weiblich,Ost,gering besiedelt


In [9]:
#this is a function that enables us to analyze for each for each subgroup how the respondents answered differently
#  to a given question/variable
def analyze_factor_patterns(dataset,variable_to_check):
    #we have a base dataframe that holds all results
    base_df=pd.DataFrame()
    #now we define the four features we want to use to check for changes in the respondents data in allbus
    factor_list=["Alter","Geschlecht","OstWest","Besiedelung"]
    #we iterate through all possible combinations of these features
    for element in factor_list:
        #print(element)
        for element2 in factor_list:
            #add we za_nr to the list of the variable to check, as it is the unique identifier for each respondent
            baselist=[variable_to_check,"za_nr"]

            #we build also a list of the features we want to check for
            extralist=[element,element2]
            #set it to remove duplicates, in case we pick the same feature twice
            extralist=list(set(extralist))

            #and combine the two lists and filter the dataset for the columns we need
            baselist.extend(extralist)
            dataset_reduced=dataset[baselist]
            #remove all rows with missing values
            dataset_reduced = dataset_reduced[~dataset_reduced.isin(["nan"]).any(axis=1)]

            #we also define a extralist plus which we need to group by the dataset
            #it is the name of the features we want to check for plus the variable
            # and then we count
            extralist_plus_factor=extralist+[variable_to_check]
            grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
            #we remove all columns that have missing values
            grouped_df=grouped_df.dropna(axis=1)
            #and then we calculate the share of the respondents for each subgroup by using the sum of the group
            grouped_df['percent'] = grouped_df['za_nr']*100 / grouped_df.groupby(by=extralist)['za_nr'].transform('sum')
            grouped_df['absolute'] = grouped_df['za_nr']
            
            #and we add this to the base dataframe that holds all results
            grouped_df=grouped_df.reset_index()
            base_df=pd.concat([base_df,grouped_df],axis=0)
    #at the end we drop duplicates
    base_df=base_df.drop_duplicates()
    #reduce the dataset
    base_df_cols=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', variable_to_check,
       'absolute','percent']
    #and return the dataset
    return base_df[base_df_cols]

In [14]:
# now we do this whole procedure for variable va03 which is for inflation
factor_to_check="mp02"
df_2023_factor=analyze_factor_patterns(df_2023,factor_to_check)
df_2023_factor=df_2023_factor.sort_values(by="absolute",ascending=False)
df_2023_factor

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_54698/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_54698/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_54698/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,mp02,absolute,percent
6,NaN,NaN,NaN,West,..,3944,72.740686
3,NaN,NaN,Weiblich,NaN,..,2514,71.420455
0,NaN,NaN,Männlich,NaN,..,2380,71.708346
6,NaN,mittlere Besiedlungsdichte,NaN,NaN,..,2280,73.359073
15,NaN,NaN,Weiblich,West,..,2053,73.216833
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,Berlin,STIMME GAR NICHT ZU,0,NaN
9,NaN,gering besiedelt,NaN,Berlin,..,0,NaN
20,U30,NaN,NaN,Berlin,STIMME VOLL+GANZ ZU,0,0.000000
18,NaN,mittlere Besiedlungsdichte,NaN,Berlin,..,0,NaN


In [18]:
df_2021.columns[df_2021.columns.str.contains("mp")]

Index(['mp16', 'mp17', 'mp18', 'mp19'], dtype='object')

In [17]:
for col in df_2021.columns.stolist():
    print(col)

za_nr
doi
version
respid
substudy
mode
splt21
eastwest
german
ep01
ep03
ep04
ep06
lm01
lm02
lm19
lm20
lm21
lm22
lm14
xr19
xr20
lm27
lm28
lm29
lm30
lm31
lm32
lm33
lm34
lm35
lm36
lm37
lm38
lm39
la01
id02
id01
mi05
mi06
mi07
mi08
mi09
mi10
mi11
sex
mborn
yborn
age
agec
dn07
dm02
dm02c
dm03
dg10
dg03
dm06
dn01
dn02
dn04
dn05
ma01b
ma02
ma03
ma04
mc01
mc02
mc03
mc04
pn11
fr07
fr08
fr03b
fr04b
fr05b
fr09
fr10
fr11
fr12
fe13
fe14
fe15
fe16
fe17
ja01
ja02
ja03
ja04
ja05
ja06
ja07
ja08
ja09
ja10
ja11
lp03
lp04
lp05
lp06
vm08
vm09
vm10
vm11
vm12
vm13
vm14
vm15
st01
pt01
pt02
pt03
pt04
pt06
pt07
pt08
pt09
pt10
pt11
pt12
pt14
pt15
pt19
pt20
ca24
cf03
im01
im17
im18
im19
im20
im21
iw04
pd11
pi07
pi01
pi02
pc01
pc02
pc03
pc04
pc05
pc06
pc07
pc08
pc09
pc10
pc11
pc17
pc19
pc20
pa02a
va01
va02
va03
va04
ingle
pa01
ps03
ca01
ca02
ca03
ca04
ca05
ca06
ca07
ca08
ca09
ca10
ca11
ca25
ca26
ca27
ca28
ca29
ca30
ca15
ca16
ca17
ca18
ca34
ca31
ca35
ca36
cs01
cs02
cs03
cs04
cs05
cs06
cs08
cs09
cp01
cp02
cp03
cp04
c

In [15]:
# now we do this whole procedure for variable va03 which is for inflation

df_2021_factor=analyze_factor_patterns(df_2021,factor_to_check)
df_2021_factor=df_2021_factor.sort_values(by="absolute",ascending=False)
df_2021_factor

KeyError: "['mp02'] not in index"

In [24]:
# and we merge the datasets of 2021 and 2023 based on all the features
comp_factor=pd.merge(df_2021_factor,df_2023_factor,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest',factor_to_check],suffixes=["_2021","_2023"])
comp_factor

,Alter,Besiedelung,Geschlecht,OstWest,ma01b,absolute_2021,percent_2021,absolute_2023,percent_2023
0,NaN,NaN,NaN,West,..,3473,68.867737,3876,71.289314
1,NaN,NaN,Weiblich,NaN,..,2170,64.989518,2400,67.529544
2,NaN,NaN,Männlich,NaN,..,2097,69.002962,2306,69.562594
3,NaN,mittlere Besiedlungsdichte,NaN,NaN,..,1863,66.157670,2105,67.706658
4,NaN,NaN,Weiblich,West,..,1785,67.054846,1970,69.660537
...,...,...,...,...,...,...,...,...,...
190,NaN,mittlere Besiedlungsdichte,NaN,Berlin,..,0,NaN,0,NaN
191,NaN,gering besiedelt,NaN,Berlin,STIMME VOLL ZU,0,NaN,0,NaN
192,NaN,gering besiedelt,NaN,Berlin,STIMME GAR NICHT ZU,0,NaN,0,NaN
193,NaN,gering besiedelt,NaN,Berlin,..,0,NaN,0,NaN


In [12]:
non_berlin_comp=comp_factor[comp_factor["OstWest"]!="Berlin"]
non_berlin_comp

NameError: name 'comp_factor' is not defined

In [13]:
non_berlin_comp["diff"]=non_berlin_comp["percent_2023"]-non_berlin_comp["percent_2021"]
non_berlin_comp=non_berlin_comp.sort_values(by="diff",ascending=False)

NameError: name 'non_berlin_comp' is not defined

In [34]:
non_berlin_comp=non_berlin_comp[non_berlin_comp.ma01b=='STIMME VOLL ZU']

In [35]:
non_berlin_comp.head(30)

,Alter,Besiedelung,Geschlecht,OstWest,ma01b,absolute_2021,percent_2021,absolute_2023,percent_2023,diff
104,50-64,NaN,NaN,Ost,STIMME VOLL ZU,109,35.620915,155,47.112462,11.491547
84,NaN,NaN,Männlich,Ost,STIMME VOLL ZU,193,37.044146,263,45.739130,8.694985
74,50-64,mittlere Besiedlungsdichte,NaN,NaN,STIMME VOLL ZU,229,28.236745,309,35.640138,7.403394
101,50-64,dicht besiedelt,NaN,NaN,STIMME VOLL ZU,115,20.034843,167,27.022654,6.987811
95,NaN,gering besiedelt,NaN,Ost,STIMME VOLL ZU,153,44.347826,191,51.206434,6.858608
68,50-64,NaN,Männlich,NaN,STIMME VOLL ZU,265,29.642058,336,35.257083,5.615025
44,50-64,NaN,NaN,NaN,STIMME VOLL ZU,484,27.114846,607,31.713689,4.598843
119,NaN,dicht besiedelt,NaN,Ost,STIMME VOLL ZU,74,28.906250,90,33.210332,4.304082
50,NaN,NaN,NaN,Ost,STIMME VOLL ZU,414,39.019793,500,43.177893,4.158100
60,50-64,NaN,NaN,West,STIMME VOLL ZU,350,24.857955,437,28.655738,3.797783


In [28]:
non_berlin_comp.tail(30)

,Alter,Besiedelung,Geschlecht,OstWest,ma01b,absolute_2021,percent_2021,absolute_2023,percent_2023,diff
117,30-49,NaN,NaN,West,STIMME GAR NICHT ZU,77,5.670103,42,2.857143,-2.812960
62,NaN,gering besiedelt,NaN,West,STIMME VOLL ZU,337,34.922280,335,31.935176,-2.987103
91,NaN,dicht besiedelt,NaN,Ost,..,173,67.578125,175,64.575646,-3.002479
13,50-64,NaN,NaN,West,..,1003,71.235795,1039,68.131148,-3.104648
132,50-64,NaN,Weiblich,NaN,STIMME GAR NICHT ZU,49,5.499439,23,2.393340,-3.106099
43,NaN,mittlere Besiedlungsdichte,Weiblich,NaN,STIMME VOLL ZU,489,33.062880,481,29.820211,-3.242670
16,NaN,mittlere Besiedlungsdichte,Männlich,NaN,..,938,70.157068,1001,66.911765,-3.245303
133,30-49,NaN,Männlich,NaN,STIMME GAR NICHT ZU,47,5.292793,18,1.973684,-3.319109
158,U30,mittlere Besiedlungsdichte,NaN,NaN,STIMME GAR NICHT ZU,23,6.824926,15,3.504673,-3.320253
69,Ü65,dicht besiedelt,NaN,NaN,STIMME VOLL ZU,263,42.215088,250,38.819876,-3.395213


In [29]:
#also we do the same for party preferences
df_2021_party=analyze_factor_patterns(df_2021,"pv01")
df_2021_party

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_51133/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_51133/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_51133/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
0,30-49,NaN,NaN,NaN,AFD,137,6.135244
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,114,5.105240
2,30-49,NaN,NaN,NaN,CDU-CSU,477,21.361397
3,30-49,NaN,NaN,NaN,DIE GRUENEN,743,33.273623
4,30-49,NaN,NaN,NaN,DIE LINKE,143,6.403941
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,881,25.403691
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,200,5.767013
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,471,13.581315
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,537,15.484429


In [30]:
df_2023_party=analyze_factor_patterns(df_2023,"pv01")
df_2023_party

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_51133/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_51133/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_51133/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
0,30-49,NaN,NaN,NaN,AFD,271,12.212708
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,77,3.470032
2,30-49,NaN,NaN,NaN,CDU-CSU,426,19.197837
3,30-49,NaN,NaN,NaN,DIE GRUENEN,534,24.064894
4,30-49,NaN,NaN,NaN,DIE LINKE,138,6.219018
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,711,19.921547
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,158,4.427010
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,381,10.675259
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,721,20.201737


In [31]:
df_2023_party[df_2023_party.percent.isna()]

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
24,NaN,gering besiedelt,NaN,Berlin,AFD,0,NaN
25,NaN,gering besiedelt,NaN,Berlin,ANDERE PARTEI,0,NaN
26,NaN,gering besiedelt,NaN,Berlin,CDU-CSU,0,NaN
27,NaN,gering besiedelt,NaN,Berlin,DIE GRUENEN,0,NaN
28,NaN,gering besiedelt,NaN,Berlin,DIE LINKE,0,NaN
29,NaN,gering besiedelt,NaN,Berlin,FDP,0,NaN
30,NaN,gering besiedelt,NaN,Berlin,SPD,0,NaN
31,NaN,gering besiedelt,NaN,Berlin,WUERDE NICHT WAEHLEN,0,NaN
48,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AFD,0,NaN
49,NaN,mittlere Besiedlungsdichte,NaN,Berlin,ANDERE PARTEI,0,NaN


In [32]:
df_2021_party[df_2021_party.percent.isna()]

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
24,NaN,gering besiedelt,NaN,Berlin,AFD,0,NaN
25,NaN,gering besiedelt,NaN,Berlin,ANDERE PARTEI,0,NaN
26,NaN,gering besiedelt,NaN,Berlin,CDU-CSU,0,NaN
27,NaN,gering besiedelt,NaN,Berlin,DIE GRUENEN,0,NaN
28,NaN,gering besiedelt,NaN,Berlin,DIE LINKE,0,NaN
29,NaN,gering besiedelt,NaN,Berlin,FDP,0,NaN
30,NaN,gering besiedelt,NaN,Berlin,SPD,0,NaN
31,NaN,gering besiedelt,NaN,Berlin,WUERDE NICHT WAEHLEN,0,NaN
48,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AFD,0,NaN
49,NaN,mittlere Besiedlungsdichte,NaN,Berlin,ANDERE PARTEI,0,NaN


In [17]:
#we also merge the datasets of 2021 and 2023 based on all the features for the party preferences
comp_party=pd.merge(df_2021_party,df_2023_party,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'pv01'],suffixes=["_2021","_2023"])
comp_party["diff_percent"]=comp_party["percent_2023"]-comp_party["percent_2021"]
comp_party.sort_values("diff_percent")

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
126,50-64,NaN,NaN,Berlin,SPD,19,29.230769,7,9.459459,-19.771310
157,U30,NaN,NaN,Ost,FDP,28,22.580645,5,3.759398,-18.821247
151,U30,NaN,NaN,Berlin,WUERDE NICHT WAEHLEN,9,18.000000,0,0.000000,-18.000000
170,Ü65,NaN,NaN,Berlin,CDU-CSU,30,35.294118,17,18.279570,-17.014548
146,U30,NaN,NaN,Berlin,CDU-CSU,8,16.000000,0,0.000000,-16.000000
...,...,...,...,...,...,...,...,...,...,...
475,NaN,mittlere Besiedlungsdichte,NaN,Berlin,DIE GRUENEN,0,NaN,0,NaN,NaN
476,NaN,mittlere Besiedlungsdichte,NaN,Berlin,DIE LINKE,0,NaN,0,NaN,NaN
477,NaN,mittlere Besiedlungsdichte,NaN,Berlin,FDP,0,NaN,0,NaN,NaN
478,NaN,mittlere Besiedlungsdichte,NaN,Berlin,SPD,0,NaN,0,NaN,NaN


In [18]:

#and we pivot it based on the party preferences
comp_party_pivot=comp_party.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='pv01')[['percent_2021','percent_2023',"absolute_2021","absolute_2023"]].reset_index()
comp_party_pivot

Alter                 Besiedelung Geschlecht OstWest percent_2021  \
pv01                                                               AFD   
0      NaN                         NaN        NaN  Berlin     2.950820   
1      NaN                         NaN        NaN     Ost    11.746522   
2      NaN                         NaN        NaN    West     4.289587   
3      NaN                         NaN   Männlich     NaN     7.808564   
4      NaN                         NaN   Männlich  Berlin     4.827586   
..     ...                         ...        ...     ...          ...   
60     Ü65                         NaN   Männlich     NaN     6.226766   
61     Ü65                         NaN   Weiblich     NaN     1.906275   
62     Ü65             dicht besiedelt        NaN     NaN     3.819918   
63     Ü65            gering besiedelt        NaN     NaN     5.040323   
64     Ü65  mittlere Besiedlungsdichte        NaN     NaN     3.435805   

                                                                 ...  \
pv01 ANDERE PARTEI    CDU-CSU DIE GRUENEN  DIE LINKE        FDP  ...   
0         3.606557  21.639344   26.885246   9.180328   7.213115  ...   
1         5.641422  23.106646   14.992272  12.905719  12.210201  ...   
2         3.412534  25.912933   29.835752   5.469622  11.880083  ...   
3         3.803526  24.055416   24.382872   7.506297  13.173804  ...   
4         2.068966  17.931034   27.586207   7.586207  12.413793  ...   
..             ...        ...         ...        ...        ...  ...   
60        0.743494  30.762082   14.962825   5.947955  13.661710  ...   
61        1.985703  36.219222   17.553614   6.592534  10.484512  ...   
62        1.227831  30.422920   17.462483   4.911323   8.867667  ...   
63        2.016129  35.483871   13.709677   8.266129  11.693548  ...   
64        1.265823  35.081374   16.817360   6.329114  14.104882  ...   

     absolute_2021                      absolute_2023                        \
pv01           SPD WUERDE NICHT WAEHLEN           AFD ANDERE PARTEI CDU-CSU   
0               60                   27            22             6      45   
1              159                   92           294            49     267   
2              961                  243           480           215    1609   
3              618                  147           526           156     940   
4               27                   13            15             3      20   
..             ...                  ...           ...           ...     ...   
60             267                   31            88             8     385   
61             247                   71            57            23     426   
62             202                   42            32             6     229   
63              96                   22            46            13     193   
64             216                   38            67            12     389   

                                                            
pv01 DIE GRUENEN DIE LINKE  FDP   SPD WUERDE NICHT WAEHLEN  
0             84        78   19    60                   23  
1            146       140   77   252                   70  
2           1518       240  641  1389                  318  
3            760       225  442   816                  214  
4             36        41    7    29                   12  
..           ...       ...  ...   ...                  ...  
60           154        54   92   321                   36  
61           219        41   94   388                   52  
62           101        37   84   292                   34  
63            65        24   22   117                   17  
64           207        34   80   300                   37  

[65 rows x 36 columns]

In [19]:
# to remove the double index ot the columns, we bring them to one name in one line
new_columns=[]
for index in comp_party_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_party_pivot.columns=new_columns
comp_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,DIE LINKE_percent_2021,FDP_percent_2021,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,Berlin,2.950820,3.606557,21.639344,26.885246,9.180328,7.213115,...,60,27,22,6,45,84,78,19,60,23
1,NaN,NaN,NaN,Ost,11.746522,5.641422,23.106646,14.992272,12.905719,12.210201,...,159,92,294,49,267,146,140,77,252,70
2,NaN,NaN,NaN,West,4.289587,3.412534,25.912933,29.835752,5.469622,11.880083,...,961,243,480,215,1609,1518,240,641,1389,318
3,NaN,NaN,Männlich,NaN,7.808564,3.803526,24.055416,24.382872,7.506297,13.173804,...,618,147,526,156,940,760,225,442,816,214
4,NaN,NaN,Männlich,Berlin,4.827586,2.068966,17.931034,27.586207,7.586207,12.413793,...,27,13,15,3,20,36,41,7,29,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,6.226766,0.743494,30.762082,14.962825,5.947955,13.661710,...,267,31,88,8,385,154,54,92,321,36
61,Ü65,NaN,Weiblich,NaN,1.906275,1.985703,36.219222,17.553614,6.592534,10.484512,...,247,71,57,23,426,219,41,94,388,52
62,Ü65,dicht besiedelt,NaN,NaN,3.819918,1.227831,30.422920,17.462483,4.911323,8.867667,...,202,42,32,6,229,101,37,84,292,34
63,Ü65,gering besiedelt,NaN,NaN,5.040323,2.016129,35.483871,13.709677,8.266129,11.693548,...,96,22,46,13,193,65,24,22,117,17


In [20]:
comp_inflation

,Alter,Geschlecht,OstWest,Besiedelung,absolute_2021,percent_2021,absolute_2023,percent_2023
0,30-49,Männlich,NaN,NaN,227,24.620390,471,33.933718
1,30-49,Weiblich,NaN,NaN,200,22.497188,639,46.950771
2,30-49,NaN,Berlin,NaN,24,28.915663,54,41.538462
3,30-49,NaN,Ost,NaN,75,23.809524,199,44.320713
4,30-49,NaN,West,NaN,328,23.213022,857,39.493088
...,...,...,...,...,...,...,...,...
60,NaN,NaN,West,mittlere Besiedlungsdichte,383,16.515740,1246,34.620728
61,NaN,NaN,West,NaN,929,18.598599,2518,32.540708
62,NaN,NaN,NaN,dicht besiedelt,434,19.523167,1031,30.225740
63,NaN,NaN,NaN,gering besiedelt,303,22.411243,717,35.112635


In [21]:
#and then again we merge both datasets
comp_inflation_party_pivot=pd.merge(comp_inflation,comp_party_pivot,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])
comp_inflation_party_pivot

,Alter,Geschlecht,OstWest,Besiedelung,absolute_2021,percent_2021,absolute_2023,percent_2023,AFD_percent_2021,ANDERE PARTEI_percent_2021,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,30-49,Männlich,NaN,NaN,227,24.620390,471,33.933718,9.034544,4.605846,...,105,44,173,51,236,209,65,131,225,93
1,30-49,Weiblich,NaN,NaN,200,22.497188,639,46.950771,3.170290,5.615942,...,119,59,98,26,190,325,73,95,169,60
2,30-49,NaN,Berlin,NaN,24,28.915663,54,41.538462,3.809524,10.476190,...,10,6,14,2,7,39,24,4,15,8
3,30-49,NaN,Ost,NaN,75,23.809524,199,44.320713,11.764706,8.683473,...,29,23,92,18,52,59,39,30,37,28
4,30-49,NaN,West,NaN,328,23.213022,857,39.493088,5.138340,4.065500,...,185,74,165,57,367,436,75,192,342,117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,NaN,NaN,West,mittlere Besiedlungsdichte,383,16.515740,1246,34.620728,4.245122,3.183841,...,469,101,253,115,781,653,99,347,599,143
61,NaN,NaN,West,NaN,929,18.598599,2518,32.540708,4.289587,3.412534,...,961,243,480,215,1609,1518,240,641,1389,318
62,NaN,NaN,NaN,dicht besiedelt,434,19.523167,1031,30.225740,4.021739,3.079710,...,407,121,189,68,523,760,220,218,710,156
63,NaN,NaN,NaN,gering besiedelt,303,22.411243,717,35.112635,7.856273,4.872107,...,236,108,238,59,485,277,80,138,270,82


In [22]:
comp_inflation_party_pivot.columns

Index(['Alter', 'Geschlecht', 'OstWest', 'Besiedelung', 'absolute_2021',
       'percent_2021', 'absolute_2023', 'percent_2023', 'AFD_percent_2021',
       'ANDERE PARTEI_percent_2021', 'CDU-CSU_percent_2021',
       'DIE GRUENEN_percent_2021', 'DIE LINKE_percent_2021',
       'FDP_percent_2021', 'SPD_percent_2021',
       'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
       'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_2023',
       'DIE GRUENEN_percent_2023', 'DIE LINKE_percent_2023',
       'FDP_percent_2023', 'SPD_percent_2023',
       'WUERDE NICHT WAEHLEN_percent_2023', 'AFD_absolute_2021',
       'ANDERE PARTEI_absolute_2021', 'CDU-CSU_absolute_2021',
       'DIE GRUENEN_absolute_2021', 'DIE LINKE_absolute_2021',
       'FDP_absolute_2021', 'SPD_absolute_2021',
       'WUERDE NICHT WAEHLEN_absolute_2021', 'AFD_absolute_2023',
       'ANDERE PARTEI_absolute_2023', 'CDU-CSU_absolute_2023',
       'DIE GRUENEN_absolute_2023', 'DIE LINKE_absolute_2023',
       'FDP_absol

In [23]:
#we take in the data from the zensus that gives a share of the population for each subgroup
#but only with two features defined as the same time
zensus_two_level=pd.read_csv("json_none.csv")
zensus_two_level

,Geschlecht,Besiedelung,OstWest,Alter,Share
0,NaN,NaN,Ost,U30,1.675804
1,NaN,gering besiedelt,NaN,U30,2.543912
2,NaN,dicht besiedelt,Ost,NaN,3.853220
3,NaN,NaN,Ost,30-49,4.559863
4,NaN,NaN,Ost,50-64,4.751534
5,NaN,NaN,Ost,Ü65,5.482466
6,NaN,gering besiedelt,Ost,NaN,5.491600
7,NaN,gering besiedelt,NaN,30-49,5.723338
8,NaN,mittlere Besiedlungsdichte,NaN,U30,5.842770
9,NaN,dicht besiedelt,NaN,U30,6.005464


In [24]:
# we merge this
comp_inflation_party_pivot_zensus=pd.merge(comp_inflation_party_pivot,zensus_two_level,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])


In [25]:
#and we filter only for the percentage results  
perc_share_df=comp_inflation_party_pivot_zensus[comp_inflation_party_pivot_zensus.columns[~comp_inflation_party_pivot_zensus.columns.str.contains("absolute")]]

In [26]:
perc_share_df

,Alter,Geschlecht,OstWest,Besiedelung,percent_2021,percent_2023,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,30-49,Männlich,NaN,NaN,24.620390,33.933718,9.034544,4.605846,20.814880,31.355182,...,3.897254,14.623838,4.311074,19.949281,17.666948,5.494505,11.073542,19.019442,7.861369,13.956047
1,30-49,Weiblich,NaN,NaN,22.497188,46.950771,3.170290,5.615942,21.920290,35.235507,...,5.344203,9.459459,2.509653,18.339768,31.370656,7.046332,9.169884,16.312741,5.791506,13.845423
2,30-49,NaN,Ost,NaN,23.809524,44.320713,11.764706,8.683473,17.647059,21.288515,...,6.442577,25.915493,5.070423,14.647887,16.619718,10.985915,8.450704,10.422535,7.887324,4.559863
3,30-49,NaN,West,NaN,23.213022,39.493088,5.138340,4.065500,22.981366,35.121400,...,4.178430,9.423187,3.255283,20.959452,24.900057,4.283267,10.965163,19.531696,6.681896,21.949289
4,30-49,NaN,NaN,dicht besiedelt,21.661721,35.449219,3.460208,4.498270,13.148789,49.365629,...,3.344867,6.198830,4.327485,12.280702,34.152047,8.187135,7.602339,23.625731,3.625731,10.287935
5,30-49,NaN,NaN,gering besiedelt,30.456853,46.057348,8.590308,5.506608,31.277533,17.180617,...,5.506608,21.296296,1.620370,28.009259,14.583333,5.555556,9.490741,9.259259,10.185185,5.723338
6,30-49,NaN,NaN,mittlere Besiedlungsdichte,21.668910,41.988003,7.456140,5.482456,24.232456,25.986842,...,5.372807,13.519313,3.540773,21.459227,19.206009,4.721030,12.875536,16.309013,8.369099,11.790197
7,30-49,NaN,NaN,NaN,23.578134,40.378319,6.135244,5.105240,21.361397,33.273623,...,4.612629,12.212708,3.470032,19.197837,24.064894,6.219018,10.184768,17.755746,6.894998,27.801470
8,50-64,Männlich,NaN,NaN,12.799168,24.266285,6.613757,3.350970,29.453263,22.398589,...,3.968254,17.500000,6.034483,20.775862,20.517241,5.086207,10.000000,15.086207,5.000000,14.218163
9,50-64,Weiblich,NaN,NaN,16.647792,31.879914,5.320197,3.842365,25.024631,34.088670,...,4.827586,7.784986,4.541242,26.969416,21.964782,5.468026,6.209453,20.481928,6.580167,14.435372


In [27]:
perc_share_df=perc_share_df.round(1)
perc_share_df


,Alter,Geschlecht,OstWest,Besiedelung,percent_2021,percent_2023,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,30-49,Männlich,NaN,NaN,24.6,33.9,9.0,4.6,20.8,31.4,...,3.9,14.6,4.3,19.9,17.7,5.5,11.1,19.0,7.9,14.0
1,30-49,Weiblich,NaN,NaN,22.5,47.0,3.2,5.6,21.9,35.2,...,5.3,9.5,2.5,18.3,31.4,7.0,9.2,16.3,5.8,13.8
2,30-49,NaN,Ost,NaN,23.8,44.3,11.8,8.7,17.6,21.3,...,6.4,25.9,5.1,14.6,16.6,11.0,8.5,10.4,7.9,4.6
3,30-49,NaN,West,NaN,23.2,39.5,5.1,4.1,23.0,35.1,...,4.2,9.4,3.3,21.0,24.9,4.3,11.0,19.5,6.7,21.9
4,30-49,NaN,NaN,dicht besiedelt,21.7,35.4,3.5,4.5,13.1,49.4,...,3.3,6.2,4.3,12.3,34.2,8.2,7.6,23.6,3.6,10.3
5,30-49,NaN,NaN,gering besiedelt,30.5,46.1,8.6,5.5,31.3,17.2,...,5.5,21.3,1.6,28.0,14.6,5.6,9.5,9.3,10.2,5.7
6,30-49,NaN,NaN,mittlere Besiedlungsdichte,21.7,42.0,7.5,5.5,24.2,26.0,...,5.4,13.5,3.5,21.5,19.2,4.7,12.9,16.3,8.4,11.8
7,30-49,NaN,NaN,NaN,23.6,40.4,6.1,5.1,21.4,33.3,...,4.6,12.2,3.5,19.2,24.1,6.2,10.2,17.8,6.9,27.8
8,50-64,Männlich,NaN,NaN,12.8,24.3,6.6,3.4,29.5,22.4,...,4.0,17.5,6.0,20.8,20.5,5.1,10.0,15.1,5.0,14.2
9,50-64,Weiblich,NaN,NaN,16.6,31.9,5.3,3.8,25.0,34.1,...,4.8,7.8,4.5,27.0,22.0,5.5,6.2,20.5,6.6,14.4


In [28]:
perc_share_df["Geschlecht"]=perc_share_df["Geschlecht"].str.replace("Männlich","Maennlich")
perc_share_df["Alter"]=perc_share_df["Alter"].str.replace("Ü65","65plus")
perc_share_df["Besiedelung"]=perc_share_df["Besiedelung"].str.replace("dicht besiedelt","Stadt")
perc_share_df["Besiedelung"]=perc_share_df["Besiedelung"].str.replace("gering besiedelt","Dorf")
perc_share_df["Besiedelung"]=perc_share_df["Besiedelung"].str.replace("mittlere Besiedlungsdichte","Kleinstadt")

In [29]:
#and we write this into a csv file
perc_share_df.to_csv("perc_share_df.csv")

In [30]:
perc_share_df

,Alter,Geschlecht,OstWest,Besiedelung,percent_2021,percent_2023,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,30-49,Maennlich,NaN,NaN,24.6,33.9,9.0,4.6,20.8,31.4,...,3.9,14.6,4.3,19.9,17.7,5.5,11.1,19.0,7.9,14.0
1,30-49,Weiblich,NaN,NaN,22.5,47.0,3.2,5.6,21.9,35.2,...,5.3,9.5,2.5,18.3,31.4,7.0,9.2,16.3,5.8,13.8
2,30-49,NaN,Ost,NaN,23.8,44.3,11.8,8.7,17.6,21.3,...,6.4,25.9,5.1,14.6,16.6,11.0,8.5,10.4,7.9,4.6
3,30-49,NaN,West,NaN,23.2,39.5,5.1,4.1,23.0,35.1,...,4.2,9.4,3.3,21.0,24.9,4.3,11.0,19.5,6.7,21.9
4,30-49,NaN,NaN,Stadt,21.7,35.4,3.5,4.5,13.1,49.4,...,3.3,6.2,4.3,12.3,34.2,8.2,7.6,23.6,3.6,10.3
5,30-49,NaN,NaN,Dorf,30.5,46.1,8.6,5.5,31.3,17.2,...,5.5,21.3,1.6,28.0,14.6,5.6,9.5,9.3,10.2,5.7
6,30-49,NaN,NaN,Kleinstadt,21.7,42.0,7.5,5.5,24.2,26.0,...,5.4,13.5,3.5,21.5,19.2,4.7,12.9,16.3,8.4,11.8
7,30-49,NaN,NaN,NaN,23.6,40.4,6.1,5.1,21.4,33.3,...,4.6,12.2,3.5,19.2,24.1,6.2,10.2,17.8,6.9,27.8
8,50-64,Maennlich,NaN,NaN,12.8,24.3,6.6,3.4,29.5,22.4,...,4.0,17.5,6.0,20.8,20.5,5.1,10.0,15.1,5.0,14.2
9,50-64,Weiblich,NaN,NaN,16.6,31.9,5.3,3.8,25.0,34.1,...,4.8,7.8,4.5,27.0,22.0,5.5,6.2,20.5,6.6,14.4


In [31]:
json_array=[]
for index, row in perc_share_df.iterrows():
    print(row)
    base_dict=row[["Alter","Besiedelung","Geschlecht","OstWest","Share"]].to_dict()
    base_dict["inflation"]=row[[ 'percent_2021', 'percent_2023']].to_dict()
    base_dict["wahl"]=row[['AFD_percent_2021', 'ANDERE PARTEI_percent_2021',
        'CDU-CSU_percent_2021', 'DIE GRUENEN_percent_2021',
        'DIE LINKE_percent_2021', 'FDP_percent_2021', 'SPD_percent_2021',
        'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
        'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_2023',
        'DIE GRUENEN_percent_2023', 'DIE LINKE_percent_2023',
        'FDP_percent_2023', 'SPD_percent_2023',
        'WUERDE NICHT WAEHLEN_percent_2023']].to_dict()
    json_array.append(base_dict)
json_array[0]

Alter                                    30-49
Geschlecht                           Maennlich
OstWest                                    NaN
Besiedelung                                NaN
percent_2021                              24.6
percent_2023                              33.9
AFD_percent_2021                           9.0
ANDERE PARTEI_percent_2021                 4.6
CDU-CSU_percent_2021                      20.8
DIE GRUENEN_percent_2021                  31.4
DIE LINKE_percent_2021                     8.1
FDP_percent_2021                          12.9
SPD_percent_2021                           9.3
WUERDE NICHT WAEHLEN_percent_2021          3.9
AFD_percent_2023                          14.6
ANDERE PARTEI_percent_2023                 4.3
CDU-CSU_percent_2023                      19.9
DIE GRUENEN_percent_2023                  17.7
DIE LINKE_percent_2023                     5.5
FDP_percent_2023                          11.1
SPD_percent_2023                          19.0
WUERDE NICHT 

{'Alter': '30-49',
 'Besiedelung': nan,
 'Geschlecht': 'Maennlich',
 'OstWest': nan,
 'Share': 14.0,
 'inflation': {'percent_2021': 24.6, 'percent_2023': 33.9},
 'wahl': {'AFD_percent_2021': 9.0,
  'ANDERE PARTEI_percent_2021': 4.6,
  'CDU-CSU_percent_2021': 20.8,
  'DIE GRUENEN_percent_2021': 31.4,
  'DIE LINKE_percent_2021': 8.1,
  'FDP_percent_2021': 12.9,
  'SPD_percent_2021': 9.3,
  'WUERDE NICHT WAEHLEN_percent_2021': 3.9,
  'AFD_percent_2023': 14.6,
  'ANDERE PARTEI_percent_2023': 4.3,
  'CDU-CSU_percent_2023': 19.9,
  'DIE GRUENEN_percent_2023': 17.7,
  'DIE LINKE_percent_2023': 5.5,
  'FDP_percent_2023': 11.1,
  'SPD_percent_2023': 19.0,
  'WUERDE NICHT WAEHLEN_percent_2023': 7.9}}

In [32]:
! pip3 install simplejson


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [33]:
import simplejson as json
#And then we write out the file as a json file
with open('data_for_viz_2.json', 'w', encoding='utf-8') as file:
    json.dump(json_array, file, ignore_nan=True, indent=2)


In [34]:
comp_no_berlin_party=comp_party[comp_party["OstWest"]!="Berlin"].sort_values(by=["diff_percent"])
comp_no_berlin_party.head(60)

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
157,U30,NaN,NaN,Ost,FDP,28,22.580645,5,3.759398,-18.821247
195,30-49,dicht besiedelt,NaN,NaN,DIE GRUENEN,428,49.365629,292,34.152047,-15.213582
35,30-49,NaN,Männlich,NaN,DIE GRUENEN,354,31.355182,209,17.666948,-13.688233
235,50-64,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,325,32.146390,194,18.671800,-13.474590
243,U30,dicht besiedelt,NaN,NaN,DIE GRUENEN,262,50.384615,183,37.500000,-12.884615
355,NaN,dicht besiedelt,Männlich,NaN,DIE GRUENEN,493,36.437546,337,24.123121,-12.314425
59,50-64,NaN,Weiblich,NaN,DIE GRUENEN,346,34.088670,237,21.964782,-12.123888
443,NaN,dicht besiedelt,NaN,West,DIE GRUENEN,860,39.907193,619,28.123580,-11.783612
249,U30,gering besiedelt,NaN,NaN,ANDERE PARTEI,24,12.371134,3,1.612903,-10.758231
499,NaN,dicht besiedelt,NaN,NaN,DIE GRUENEN,1024,37.101449,760,26.722925,-10.378524


In [35]:
comp_no_berlin_party.tail(60)

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
392,NaN,mittlere Besiedlungsdichte,Weiblich,NaN,AFD,43,2.489867,135,7.674815,5.184948
250,U30,gering besiedelt,NaN,NaN,CDU-CSU,28,14.432990,37,19.892473,5.459483
46,30-49,NaN,Weiblich,NaN,SPD,119,10.778986,169,16.312741,5.533756
136,50-64,NaN,NaN,West,AFD,72,4.183614,175,9.754738,5.571124
32,30-49,NaN,Männlich,NaN,AFD,102,9.034544,173,14.623838,5.589294
223,50-64,dicht besiedelt,NaN,NaN,WUERDE NICHT WAEHLEN,28,4.375000,69,10.058309,5.683309
216,50-64,dicht besiedelt,NaN,NaN,AFD,41,6.406250,83,12.099125,5.692875
190,Ü65,NaN,NaN,West,SPD,411,22.833333,550,28.810896,5.977562
208,30-49,mittlere Besiedlungsdichte,NaN,NaN,AFD,68,7.456140,126,13.519313,6.063173
0,30-49,NaN,NaN,NaN,AFD,137,6.135244,271,12.212708,6.077464


In [36]:
comp_inflation
comp_inflation["diff_percent"]=comp_inflation["percent_2023"]-comp_inflation["percent_2021"]
comp_no_berlin_infl=comp_inflation[comp_inflation["OstWest"]!="Berlin"]

comp_no_berlin_infl=comp_no_berlin_infl.sort_values("diff_percent")

In [37]:
comp_no_berlin_infl.head(20)

,Alter,Geschlecht,OstWest,Besiedelung,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
32,Ü65,NaN,NaN,dicht besiedelt,134,22.558923,164,18.021978,-4.536945
40,NaN,Männlich,NaN,gering besiedelt,165,23.605150,289,28.557312,4.952162
27,Ü65,Männlich,NaN,NaN,149,18.104496,285,23.284314,5.179818
31,Ü65,NaN,West,NaN,237,17.516630,511,23.856209,6.339579
35,Ü65,NaN,NaN,NaN,317,17.889391,697,25.135233,7.245842
15,50-64,NaN,NaN,gering besiedelt,79,18.202765,167,26.012461,7.809696
58,NaN,NaN,West,dicht besiedelt,338,19.719953,747,28.306177,8.586223
28,Ü65,Weiblich,NaN,NaN,168,17.702845,412,26.597805,8.894960
0,30-49,Männlich,NaN,NaN,227,24.620390,471,33.933718,9.313327
39,NaN,Männlich,NaN,dicht besiedelt,185,17.485822,451,27.516779,10.030956


In [38]:
comp_no_berlin_infl.tail(20)

,Alter,Geschlecht,OstWest,Besiedelung,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
45,NaN,Weiblich,West,NaN,471,19.053398,1416,35.685484,16.632086
49,NaN,Weiblich,NaN,NaN,639,20.050204,1838,36.745302,16.695098
8,30-49,NaN,NaN,NaN,427,23.578134,1110,40.378319,16.800186
24,U30,NaN,NaN,gering besiedelt,43,28.666667,115,45.634921,16.968254
64,NaN,NaN,NaN,mittlere Besiedlungsdichte,485,17.414722,1476,34.429671,17.014949
60,NaN,NaN,West,mittlere Besiedlungsdichte,383,16.515740,1246,34.620728,18.104988
44,NaN,Weiblich,Ost,NaN,123,21.964286,335,40.458937,18.494651
48,NaN,Weiblich,NaN,mittlere Besiedlungsdichte,252,18.407597,830,37.727273,19.319676
7,30-49,NaN,NaN,mittlere Besiedlungsdichte,161,21.668910,490,41.988003,20.319094
47,NaN,Weiblich,NaN,gering besiedelt,138,21.133231,428,41.553398,20.420167


In [39]:
comp_no_berlin_infl

,Alter,Geschlecht,OstWest,Besiedelung,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
32,Ü65,NaN,NaN,dicht besiedelt,134,22.558923,164,18.021978,-4.536945
40,NaN,Männlich,NaN,gering besiedelt,165,23.605150,289,28.557312,4.952162
27,Ü65,Männlich,NaN,NaN,149,18.104496,285,23.284314,5.179818
31,Ü65,NaN,West,NaN,237,17.516630,511,23.856209,6.339579
35,Ü65,NaN,NaN,NaN,317,17.889391,697,25.135233,7.245842
15,50-64,NaN,NaN,gering besiedelt,79,18.202765,167,26.012461,7.809696
58,NaN,NaN,West,dicht besiedelt,338,19.719953,747,28.306177,8.586223
28,Ü65,Weiblich,NaN,NaN,168,17.702845,412,26.597805,8.894960
0,30-49,Männlich,NaN,NaN,227,24.620390,471,33.933718,9.313327
39,NaN,Männlich,NaN,dicht besiedelt,185,17.485822,451,27.516779,10.030956
